## Tensorflow on Char-RNN
#### This program was created based on the following links: 
https://github.com/patrickcnkm/tensorflow-char-rnn


####  The target of this coding is:
- Understanding how the weights and biases are being produced and changed in each layer, then get more sense how the RNN is working, and consequently get further understanding on GRU and LSTM.

Since this code will be visualized by tensorboard, it is proposed to start tensorboard in the folder where this code is located in the command windows by the following command:
> tensorboard --logdir=logs/

The usage of tensorboard can be found in the following link:
https://www.analyticsvidhya.com/blog/2017/07/debugging-neural-network-with-tensorboard/

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import itertools
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
from time import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

import logging
# Disable Tensorflow logging messages.
#logging.getLogger('tensorflow').setLevel(logging.WARNING)

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


You have TensorFlow version 1.12.0


### Initialize the parameters of model

In [3]:
use_batch=True
batch_size = 20
model='rnn'
num_unrollings = 10
is_training=True
if not use_batch:
    batch_size = 1
    num_unrollings = 1
hidden_size = 128
#vocab_size = 1000
max_grad_norm = 5
num_layers = 2
embedding_size = 0
dropout = 0.0
input_dropout = 0.0
num_epochs = 50
valid_frac = 0.05
train_frac = 0.05
learning_rate = 2e-3
decay_rate = 0.95
verbose = 0
best_valid_ppl = np.inf
encoding = 'utf-8'

### vocabulary wrangling

In [4]:
def create_vocab(text):
    unique_chars = list(set(text))
    vocab_size = len(unique_chars)
    vocab_index_dict = {}
    index_vocab_dict = {}
    for i, char in enumerate(unique_chars):
        vocab_index_dict[char] = i
        index_vocab_dict[i] = char
    return vocab_index_dict, index_vocab_dict, vocab_size


def load_vocab(vocab_file, encoding):
    with codecs.open(vocab_file, 'r', encoding=encoding) as f:
        vocab_index_dict = json.load(f)
    index_vocab_dict = {}
    vocab_size = 0
    for char, index in iteritems(vocab_index_dict):
        index_vocab_dict[index] = char
        vocab_size += 1
    return vocab_index_dict, index_vocab_dict, vocab_size


def save_vocab(vocab_index_dict, vocab_file, encoding):
    with codecs.open(vocab_file, 'w', encoding=encoding) as f:
        json.dump(vocab_index_dict, f, indent=2, sort_keys=True)

In [5]:
# Create vocabulary based on given text
text='I like apple'
logging.info('Creating vocabulary')
vocab_index_dict, index_vocab_dict, vocab_size = create_vocab(text)
#params['vocab_size'] = vocab_size
logging.info('Vocab size: %d', vocab_size)

In [6]:
if embedding_size <= 0:
    input_size = vocab_size
    # Don't do dropout on one hot representation
    input_dropout = 0.0
else:
    input_size = embedding_size
    model_size = (embedding_size * vocab_size + # embedding parameters
                       # lstm parameters
                       4 * hidden_size * (hidden_size + input_size + 1) +
                       # softmax parameters
                       vocab_size * (hidden_size + 1) +
                       # multilayer lstm parameters for extra layers.
                       (num_layers - 1) * 4 * hidden_size *
                       (hidden_size + hidden_size + 1))
    # self.decay_rate = decay_rate

In [7]:
params = {'batch_size': batch_size,
          'num_unrollings': num_unrollings,
          'hidden_size': hidden_size,
          'max_grad_norm': max_grad_norm,
          'embedding_size': embedding_size,
          'num_layers': num_layers,
          'learning_rate': learning_rate,
          'model': model,
          'dropout': dropout,
          'input_dropout': input_dropout,
          'vocab_size':vocab_size}

### Create the utility functions

In [8]:
def batches2string(batches, index_vocab_dict):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, id2char_list(b, index_vocab_dict))]
  return s


def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]


def char2id(char, vocab_index_dict):
  try:
    return vocab_index_dict[char]
  except KeyError:
    logging.info('Unexpected char %s', char)
    return 0


def id2char(index, index_vocab_dict):
  return index_vocab_dict[index]

    
def id2char_list(lst, index_vocab_dict):
  return [id2char(i, index_vocab_dict) for i in lst]


def create_tuple_placeholders_with_default(inputs, extra_dims, shape):

  if isinstance(shape, int):
    result = tf.placeholder_with_default(
      inputs, list(extra_dims) + [shape])
  else:
    subplaceholders = [create_tuple_placeholders_with_default(
      subinputs, extra_dims, subshape)
                       for subinputs, subshape in zip(inputs, shape)]
    t = type(shape)
    if t == tuple:
      result = t(subplaceholders)
    else:
      result = t(*subplaceholders)    
  return result

        
def create_tuple_placeholders(dtype, extra_dims, shape):
  if isinstance(shape, int):
    result = tf.placeholder(dtype, list(extra_dims) + [shape])
  else:
    subplaceholders = [create_tuple_placeholders(dtype, extra_dims, subshape)
                       for subshape in shape]
    t = type(shape)

    # Handles both tuple and LSTMStateTuple.
    if t == tuple:
      result = t(subplaceholders)
    else:
      result = t(*subplaceholders)
  return result
  

### Create the placeholders

In [9]:
# Placeholder to feed in input and targets/labels data.
input_data = tf.placeholder(tf.int64,
                            [batch_size, num_unrollings],
                            name='inputs')
targets = tf.placeholder(tf.int64,
                         [batch_size, num_unrollings],
                         name='targets')
if model == 'rnn':
    cell_fn = tf.contrib.rnn.BasicRNNCell
elif model == 'lstm':
    cell_fn = tf.contrib.rnn.BasicLSTMCell
elif model == 'gru':
    cell_fn = tf.contrib.rnn.GRUCell

# params = {'input_size': input_size}
params_1 = {}
if model == 'lstm':
    # add bias to forget gate in lstm.
    params_1['forget_bias'] = 0.0
    params_1['state_is_tuple'] = True
    # Create multilayer cell.
cell = cell_fn(
    hidden_size, reuse=tf.get_variable_scope().reuse,
    **params_1)

cells = [cell]
# params['input_size'] = self.hidden_size
# more explicit way to create cells for MultiRNNCell than
# [higher_layer_cell] * (num_layers - 1)
for i in range(num_layers-1):
    higher_layer_cell = cell_fn(
        hidden_size, reuse=tf.get_variable_scope().reuse,
        **params_1)
    cells.append(higher_layer_cell)

if is_training and dropout > 0:
    cells = [tf.contrib.rnn.DropoutWrapper(
        cell,
        output_keep_prob=1.0-dropout)
             for cell in cells]

multi_cell = tf.contrib.rnn.MultiRNNCell(cells)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [10]:

with tf.name_scope('initial_state'):
    zero_state = multi_cell.zero_state(batch_size, tf.float32)
    #subplaceholders=[]
        # zero_state is used to compute the intial state for cell.
    
  # Placeholder to feed in initial state.
  #initial_state = tf.placeholder(
  #    tf.float32,
  #    [batch_size, multi_cell.state_size],
  #    'initial_state')
  
    initial_state = create_tuple_placeholders_with_default(
      multi_cell.zero_state(batch_size, tf.float32),
      extra_dims=(None,),
      shape=multi_cell.state_size)      

# Embeddings layers.
with tf.name_scope('embedding_layer'):
    if embedding_size > 0:
        embedding = tf.get_variable('embedding', [vocab_size, embedding_size])
    else:
        embedding = tf.constant(np.eye(vocab_size), dtype=tf.float32)

    inputs = tf.nn.embedding_lookup(embedding, input_data)
    if is_training and input_dropout > 0:
        inputs = tf.nn.dropout(inputs, 1 - input_dropout)

with tf.name_scope('slice_inputs'):
  # Slice inputs into a list of shape [batch_size, 1] data colums.
  sliced_inputs = [tf.squeeze(input_, [1])
                   for input_ in tf.split(axis=1, num_or_size_splits=num_unrollings, value=inputs)]

# Copy cell to do unrolling and collect outputs.
outputs, final_state = tf.contrib.rnn.static_rnn(
    multi_cell, sliced_inputs,
  initial_state=initial_state)

final_state = final_state

with tf.name_scope('flatten_ouputs'):
  # Flatten the outputs into one dimension.
  flat_outputs = tf.reshape(tf.concat(axis=1, values=outputs), [-1, hidden_size])

with tf.name_scope('flatten_targets'):
  # Flatten the targets too.
  flat_targets = tf.reshape(tf.concat(axis=1, values=targets), [-1])

# Create softmax parameters, weights and bias.
with tf.variable_scope('softmax') as sm_vs:
    softmax_w = tf.get_variable("softmax_w", [hidden_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    logits = tf.matmul(flat_outputs, softmax_w) + softmax_b
    probs = tf.nn.softmax(logits)

with tf.name_scope('loss'):
  # Compute mean cross entropy loss for each output.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=flat_targets)
    mean_loss = tf.reduce_mean(loss)

with tf.name_scope('loss_monitor'):
  # Count the number of elements and the sum of mean_loss
  # from each batch to compute the average loss.
    count = tf.Variable(1.0, name='count')
    sum_mean_loss = tf.Variable(1.0, name='sum_mean_loss')
  
    reset_loss_monitor = tf.group(sum_mean_loss.assign(0.0),
                                  count.assign(0.0),
                                  name='reset_loss_monitor')
    update_loss_monitor = tf.group(sum_mean_loss.assign(sum_mean_loss +
                                                        mean_loss),
                                   count.assign(count + 1),
                                   name='update_loss_monitor')
    with tf.control_dependencies([update_loss_monitor]):
        average_loss = sum_mean_loss / count
        ppl = tf.exp(average_loss)

# Monitor the loss.
loss_summary_name = "average loss"
ppl_summary_name = "perplexity"

average_loss_summary = tf.summary.scalar(loss_summary_name, average_loss)
ppl_summary = tf.summary.scalar(ppl_summary_name, ppl)

# Monitor the loss.
summaries = tf.summary.merge([average_loss_summary, ppl_summary],
                                  name='loss_monitor')

global_step = tf.get_variable('global_step', [],
                                   initializer=tf.constant_initializer(0.0))

learning_rate = tf.constant(learning_rate)
if is_training:
  # learning_rate = tf.train.exponential_decay(1.0, self.global_step,
  #                                            5000, 0.1, staircase=True)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(mean_loss, tvars),
                                      max_grad_norm)
  # optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  # optimizer = tf.train.RMSPropOptimizer(learning_rate, decay_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate)

    train_op = optimizer.apply_gradients(zip(grads, tvars),
                                         global_step=global_step)

INFO:tensorflow:Summary name average loss is illegal; using average_loss instead.


INFO:tensorflow:Summary name average loss is illegal; using average_loss instead.


In [11]:
def run_epoch(session, data_size, batch_generator, is_training,
            verbose=0, freq=10, summary_writer=None, debug=False, divide_by_n=1):
    """Runs the model on the given data for one full pass."""
# epoch_size = ((data_size // self.batch_size) - 1) // self.num_unrollings
    epoch_size = data_size // (batch_size * num_unrollings)
    if data_size % (batch_size * num_unrollings) != 0:
        epoch_size += 1

    if verbose > 0:
        logging.info('epoch_size: %d', epoch_size)
        logging.info('data_size: %d', data_size)
        logging.info('num_unrollings: %d', num_unrollings)
        logging.info('batch_size: %d', batch_size)

    if is_training:
        extra_op = train_op
    else:
        extra_op = tf.no_op()

    # Prepare initial state and reset the average loss
    # computation.
    state = session.run(zero_state)
    reset_loss_monitor.run()
    start_time = time.time()
    for step in range(epoch_size // divide_by_n):
        # Generate the batch and use [:-1] as inputs and [1:] as targets.
        data = batch_generator.next()
        inputs = np.array(data[:-1]).transpose()
        targets = np.array(data[1:]).transpose()

        ops = [average_loss, final_state, extra_op,
               summaries, global_step, learning_rate]

        feed_dict = {input_data: inputs, targets: targets,
                     initial_state: state}

        results = session.run(ops, feed_dict)
        average_loss, state, _, summary_str, global_step, lr = results
  
        ppl = np.exp(average_loss)
        if (verbose > 0) and ((step+1) % freq == 0):
            logging.info("%.1f%%, step:%d, perplexity: %.3f, speed: %.0f words",
                         (step + 1) * 1.0 / epoch_size * 100, step, ppl,
                         (step + 1) * batch_size * num_unrollings /
                         (time.time() - start_time))


    logging.info("Perplexity: %.3f, speed: %.0f words per sec",
                 ppl, (step + 1) * batch_size * num_unrollings /
                 (time.time() - start_time))
    return ppl, summary_str, global_step

def sample_seq(session, length, start_text, vocab_index_dict,
             index_vocab_dict, temperature=1.0, max_prob=True):
    state = session.run(zero_state)
    
    # use start_text to warm up the RNN.
    if start_text is not None and len(start_text) > 0:
        seq = list(start_text)
        for char in start_text[:-1]:
            x = np.array([[char2id(char, vocab_index_dict)]])
            state = session.run(final_state,
                                {input_data: x,
                                 initial_state: state})
        x = np.array([[char2id(start_text[-1], vocab_index_dict)]])
    else:
        vocab_size = len(vocab_index_dict.keys())
        x = np.array([[np.random.randint(0, vocab_size)]])
        seq = []
    
    for i in range(length):
        state, logits = session.run([final_state,
                                     logits],
                                    {input_data: x,
                                     initial_state: state})
        unnormalized_probs = np.exp((logits - np.max(logits)) / temperature)
        probs = unnormalized_probs / np.sum(unnormalized_probs)
        
        if max_prob:
            sample = np.argmax(probs[0])
        else:
            sample = np.random.choice(vocab_size, 1, p=probs[0])[0]

        seq.append(id2char(sample, index_vocab_dict))
        x = np.array([[sample]])
    return ''.join(seq)
  


### Create the class of batch generator

In [12]:
class BatchGenerator(object):
    """Generate and hold batches."""

    def __init__(self, text, batch_size, n_unrollings, vocab_size,
                 vocab_index_dict, index_vocab_dict):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self.vocab_size = vocab_size
        self._n_unrollings = n_unrollings
        self.vocab_index_dict = vocab_index_dict
        self.index_vocab_dict = index_vocab_dict

        segment = self._text_size // batch_size

        # number of elements in cursor list is the same as
        # batch_size.  each batch is just the collection of
        # elements in where the cursors are pointing to.
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b] = char2id(self._text[self._cursor[b]], self.vocab_index_dict)
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._n_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches